# Overview

### Clinical Study
* In this study, we collect voice recordings of people with Parkinson's disease (PD) and other neurodegenrative diseases like ataxia (AX), as well as healthy controls (HC).
* The data collection is conducted using our app, **Vocapp**, which has two interfaces, for **users** (PD, HC), and for **samplers**.
* The conductors of the study are **samplers**. Samplers can register new users. When registering a PD, the sampler also assesses the condition of the PD with clinical questionnaires like MDS-UPDRS, MoCA, etc., and documents everything in **Vocapp**. 
* After the registration, PDs will record themselves once every month, before taking medication and one hour afterwards. 
* HC and participants with other diseases will record themselves once without any further assessment or requirements.

### Vocapp
* The app has two interfaces, one for users and one for sampler. 
* When a user starts the recording exercises, the app starts a session. All the recordings and the self-report qnnrs. will be saved under this session. The filekey uploaded by a **user** has the following path: `{username}/{session}/{filename}`.
* Sampler will upload qnnrs. files for a given user, but not under any session. The filekey of a file uploaded by a **sampler** will be: `{username}/{filename}`. This includes registration, medications, MDS-UPDRS, etc. 

### Alerts
In order to collect the data effectively, we need an alerts system with the following alerts:
* During session: if a user has stopped recording in the middle of the session, i.e. didn't upload a new file for 3 minutes, the app will send a Whatsapp msg with a reminder or with suggestions how to operate the app in case it is stuck.
* Middle session: an hour after medication intake, send a reminder to record again.
* Monthly reminder: send a monthly reminder to perform a recording session. Send a reminder on a weekly basis until the user has recorded. 
* Feedbak: we want to send a feedback as a bounty for PDs that have completed the full session (before + after medication). The feedback takes the recordings before medication, extracts some voice qualities into a dictionary, plots this dictionary, and sends it as a whatsapp msg.

### Database
* The dashboard is used to track and manage the clinical study. Thus, we extract all the metadata according to the following order:
    * List all the files from the bucket (aws).
    * Extract the metadata from the filekey name **only** (`bucket.csv`).
    * Open the csv fils and extract additional metadata from them, including registration data and questionnaires (e.g. `raw.csv`).
    * Files & Sessions:
        * Update needed attributes for filekeys and merge with samplers and phone numbers databases.
        * Match session to all filekeys (sampler's files do not have a session originally).
        * Resolve sessions issues (mostly merge sessions).
        * Propagate metadata to all filekeys.
        * Save the full database (`all_files.csv`) and the sessions (`sessions.csv`).

### Dashboard
The dashbard uses the database to plot some figures:
* Distribution of attributes 
* Patients registratino over time
* Users per sampler
* Number of full/part sessions vs. session number
* Qnnrs. results
* Broken (part) sessions
* Users that need to record again. 

# List all files from the bucket

- List all filekeys from the aws bucket.
- Extract the metadata that is embedded in the file path **only**:
    - filekey
    - username
    - Entity: PD, HC, Sampler (SA)
    - time stamps
    - pattern: RECORDING, RECORDING1, FOG, UPDRS3, etc.
    - exercise: refers to RECORDING patterns, for other patterns, exercise = pattern.
    - timing: 
        - pre: first recording part of the session, before medication.
        - post: second recording part of the session, after medication.
        - healthy: session of a healthy participant; one part only.
    - onmed: did the patient *actually* take or didn't take the medication
    - onoff: does the patient feel the effect of the medication (ON) or it has already worn off (OFF)
- New filekeys are appended to the bucket csv file.

```python
def get_bucket(skip=True) -> None:
    if (not exists(Settings.BUCKET_CSV)) or (skip==False):
        bucket = pd.DataFrame(columns=Bucket.values())
        processed_filekeys = []
    else:
        bucket = pd.read_csv(Settings.BUCKET_CSV, dtype=str)
        processed_filekeys = bucket['filekey'].to_list()       
    
    filekeys = list_bucket()
    filekeys = [f for f in filekeys if f not in processed_filekeys]

    dfs = []
    for filekey in tqdm(filekeys, desc="Adding new files to database"):
        df = pd.DataFrame(columns=Bucket.values())
        df.loc[0, Bucket.FILEKEY] = filekey
        filename = filekey.split('/')[-1]
        for pattern in Patterns.values():
            if re.match(pattern.value, filename):
                username = filekey.split('/')[0]
                df.loc[0, Bucket.USERNAME] = username
                if username.startswith("hc_"):
                        df.loc[0, Bucket.ENTITY] = Entity.HC  # Ataxia will be resolved later
                elif len(username)==40:
                    df.loc[0, Bucket.ENTITY] = Entity.PD
                else:
                    df.loc[0, Bucket.ENTITY] = Entity.SA
                
                df.loc[0, Bucket.PATTERN] = pattern.name
                df.loc[0, Bucket.EXERCISE] = pattern.name.lower()
                if pattern.name not in ["REGISTRATION0", 'APKINSON']:
                    df.loc[0, Bucket.DATE] = extract_from_filename(filekey, 'date')
                    df.loc[0, Bucket.TIME] = extract_from_filename(filekey, 'time')
                    df.loc[0, Bucket.DATETIME] = extract_from_filename(filekey, 'datetime')

                if pattern.name!="UPDATE":
                    df.loc[0, Bucket.LANG] = extract_from_filename(filekey, 'language')
                
                if pattern.name in ['RECORDING', 'RECORDING1', 'FOG', 'SDQ', 'WOQ', 'UPDATE']:
                    df.loc[0, Bucket.SESSION] = filekey.split('/')[1]
                
                if pattern.name in ['UPDRS', 'UPDRS3', 'UPDRS124']:
                    df.loc[0, 'timing'] = extract_from_filename(filekey, 'timing')                    

                if pattern.name=='RECORDING':
                    df.loc[0, Bucket.EXERCISE] = extract_from_filename(filekey, 'exercise') # override
                    df.loc[0, Bucket.TIMING] = extract_from_filename(filekey, 'timing')
                    df.loc[0, Bucket.ONMED] = extract_from_filename(filekey, 'onmed')
                    df.loc[0, Bucket.ONOFF] = extract_from_filename(filekey, 'onoff')
                elif pattern.name=='RECORDING1':
                    df.loc[0, Bucket.EXERCISE] = extract_from_filename(filekey, 'exercise') # override
                    df.loc[0, Bucket.TIMING] = extract_from_filename(filekey, 'timing')
                    df.loc[0, Bucket.ONMED] = OnMed.ONMED if filekey.endswith("_on") else OnMed.NOTONMED
                    df.loc[0, Bucket.ONOFF] = FeelOnOff.UNKNOWN
                
                dfs.append(df)
                break
    if dfs:
        dfs = pd.concat(dfs, ignore_index=True)
        bucket = pd.concat([bucket, dfs], ignore_index=True)
        bucket.to_csv(Settings.BUCKET_CSV, index=False)
```

# Fetch users login credentials from the VM (EC2)

### python code I run from my jupyter
We did not keep the credentials on the VM from the beginning, so I had to concat the data with other logs to get all the phone numbers. This is the purpose of `combine_yahav_ec2()`. In later version, we can omit it.

```python
def users_data():
    print("\nDownloading credentials from ec2...")
    os_type = get_os()
    if os_type == "Linux" or os_type == "Darwin":
        run_shell_command("chmod +x src/download_users.sh")
        run_shell_command("./src/download_users.sh")
    elif os_type == "Windows":
        run_windows_shell_command("./src/download_users_windows.ps1")
    else:
        print(f"Unsupported OS: {os_type}")
        raise SystemExit
    
    combine_yahav_ec2()
    healthy_ec2()



def combine_yahav_ec2() -> None:
    pd_yahav = pd.read_csv(Settings.USERS_YAHAV_CSV, dtype=str, index_col=0)
    pd_yahav[ExtraCols.PASSWORD.value] = np.nan
    pd_ec2 = pd.read_csv(Settings.USERSPD, dtype=str)
    pd_ec2.columns = [ExtraCols.USER_PHONE.value, ExtraCols.PASSWORD.value]
    pd_ec2['username'] = pd_ec2[ExtraCols.USER_PHONE.value].apply(hash_phone_number)
    combo = pd.concat([pd_yahav, pd_ec2], ignore_index=True)

    phonesNpasswords = pd.read_csv('resources/passwords.csv', dtype=str)
    # Merge the dataframes on ExtraCols.USER_PHONE.value with an outer join to ensure all users are included
    combined_df = pd.merge(combo, phonesNpasswords, on=ExtraCols.USER_PHONE.value, how='outer', suffixes=('_users', '_passwords'))

    # Fill missing passwords in users_df with passwords from passwords_df
    combined_df[ExtraCols.PASSWORD.value] = combined_df['password_users'].combine_first(combined_df['password_passwords'])

    # Drop the now redundant columns
    combined_df.drop(columns=['password_users', 'password_passwords'], inplace=True)
    combined_df = combined_df.drop_duplicates(subset=ExtraCols.USER_PHONE.value, keep='last', ignore_index=True)
    combined_df.to_csv(Settings.USERS_EC2_CSV)



def healthy_ec2() -> None:
    hc_ec2 = pd.read_csv(Settings.USERSHC, dtype=str)
    hc_ec2.columns = [ExtraCols.USER_PHONE.value, ExtraCols.PASSWORD.value]
    hc_ec2[Bucket.USERNAME] = hc_ec2[ExtraCols.USER_PHONE.value].apply(hash_phone_number)
    hc_ec2[Bucket.USERNAME] = "hc_" + hc_ec2[Bucket.USERNAME]

    tocat = pd.read_csv(Settings.HC_PHONES_CSV, dtype=str, index_col=0)
    tocat[ExtraCols.PASSWORD.value] = np.nan
    hc_ec2 = pd.concat([hc_ec2, tocat], ignore_index=True)
    hc_ec2 = hc_ec2.drop_duplicates(subset=ExtraCols.USER_PHONE.value, keep='last', ignore_index=True)
    hc_ec2.to_csv(Settings.HC_EC2_CSV)
```

### shell script to access the VM

```shell
#!/bin/bash

# Define variables
PEM_FILE="../build-key.pem"
EC2_USER="ec2-user"
EC2_HOST="ec2-3-83-206-91.compute-1.amazonaws.com"
CONTAINER_NAME="vocabe"
REMOTE_FILE_PATH1="/data/.usershc.csv"
EC2_LOCAL_PATH1="/home/ec2-user/.usershc.csv"
LOCAL_FILE_PATH1="resources/usershc.csv"
REMOTE_FILE_PATH2="/data/.userspd.csv"
EC2_LOCAL_PATH2="/home/ec2-user/.userspd.csv"
LOCAL_FILE_PATH2="resources/userspd.csv"

# Log into AWS EC2 and copy the file from the Docker container to the EC2 instance's home directory
ssh -i $PEM_FILE $EC2_USER@$EC2_HOST << EOF
  sudo docker cp $CONTAINER_NAME:$REMOTE_FILE_PATH1 $EC2_LOCAL_PATH1
  sudo docker cp $CONTAINER_NAME:$REMOTE_FILE_PATH2 $EC2_LOCAL_PATH2
EOF

# Check if the SSH command was successful
if [ $? -eq 0 ]; then
  echo "File copied from container to EC2 instance successfully."
else
  echo "Failed to copy file from container to EC2 instance."
  exit 1
fi

# Download the file from the EC2 instance to the local machine
scp -i $PEM_FILE $EC2_USER@$EC2_HOST:$EC2_LOCAL_PATH1 $LOCAL_FILE_PATH1
scp -i $PEM_FILE $EC2_USER@$EC2_HOST:$EC2_LOCAL_PATH2 $LOCAL_FILE_PATH2

# Check if the SCP command was successful
if [ $? -eq 0 ]; then
  echo "File downloaded to local machine successfully."
else
  echo "Failed to download file to local machine."
  exit 1
fi
```

# Extract raw data
Open csv files to get the data of:
- Participants at registration
- Data updates
- Answers to questionnaires
- Medications list

```python
def get_raw_data(skip=True, print_filekey=False):
    pd.set_option('future.no_silent_downcasting', True)

    def check_exist_and_return(filepath: str, skip=skip) -> pd.DataFrame:
        if (not exists(filepath)) or (skip==False):
            return pd.DataFrame()
        else:
            return pd.read_csv(filepath, dtype=str)
    
    def put_filekey_first(df: pd.DataFrame) -> pd.DataFrame:
        if 'filekey' in df.columns:
            cols = df.columns.tolist()
            cols.remove('filekey')
            df = df[['filekey'] + cols]
        else:
            print("The DataFrame does not contain a 'filekey' column.")
        return df

    bucket = pd.read_csv(Settings.BUCKET_CSV, dtype=str)
    raw = pd.read_csv(Settings.RAW_CSV, dtype=str) if exists(Settings.RAW_CSV) else pd.DataFrame()
    if exists(Settings.RAW_CSV):
        new_filekeys = bucket.loc[~bucket[Bucket.FILEKEY].isin(raw[Bucket.FILEKEY])].copy()
    else:
        new_filekeys = bucket.copy()
    

    updrs = check_exist_and_return(Settings.UPDRS_CSV)
    moca = check_exist_and_return(Settings.MOCA_CSV)
    pdq8 = check_exist_and_return(Settings.PDQ8_CSV)
    fog = check_exist_and_return(Settings.FOG_CSV)
    sdq = check_exist_and_return(Settings.SDQ_CSV)
    woq = check_exist_and_return(Settings.WOQ_CSV)
    registration = check_exist_and_return(Settings.REGISTRATION_CSV)
    update = check_exist_and_return(Settings.UPDATE_CSV)
    medications = check_exist_and_return(Settings.MEDICATION_CSV)

    for ii,row in tqdm(new_filekeys.iterrows(), desc="Extracting raw data", total=len(new_filekeys)):
        filekey = row[Bucket.FILEKEY]
        pattern = row[Bucket.PATTERN]
        if print_filekey:
            print(filekey)
        if pattern in ["UPDRS", "UPDRS3", "UPDRS124"]:
            if Qnnrs.UPDRS1.value not in row or pd.isna(row[Qnnrs.UPDRS1.value]) or row[Qnnrs.UPDRS1.value]=='':
                df = download_csv_to_df(filekey)
                new_filekeys.loc[ii, Qnnrs.UPDRS1] = df.loc[0, UPDRS.updrs1.value].astype(int).sum()
                new_filekeys.loc[ii, Qnnrs.UPDRS2] = df.loc[0, UPDRS.updrs2.value].astype(int).sum()
                new_filekeys.loc[ii, Qnnrs.UPDRS3] = df.loc[0, UPDRS.updrs3.value].astype(int).sum()
                new_filekeys.loc[ii, Qnnrs.UPDRS4] = df.loc[0, UPDRS.updrs4.value].astype(int).sum()
                new_filekeys.loc[ii, Qnnrs.HY] = df.loc[0, UPDRS.hy.value]
                new_filekeys.loc[ii, Bucket.SAMPLER] = df.loc[0, Bucket.SAMPLER] if Bucket.SAMPLER in df else pd.NaT
                if updrs.empty or (filekey not in updrs[Bucket.FILEKEY].tolist()):
                    df[Bucket.FILEKEY] = filekey
                    df = put_filekey_first(df)
                    updrs = pd.concat([updrs, df], ignore_index=True)
        
        if pattern=="MOCA":
            if Qnnrs.MOCA.value not in row or pd.isna(row[Qnnrs.MOCA.value]) or row[Qnnrs.MOCA.value]=='':
                df = download_csv_to_df(filekey)
                df = df.replace({"True": 1, "False": 0})
                new_filekeys.loc[ii, Qnnrs.MOCA] = df.loc[0, MoCA.moca.value].astype(int).sum()
                new_filekeys.loc[ii, Bucket.SAMPLER] = df.loc[0, Bucket.SAMPLER] if Bucket.SAMPLER in df else pd.NaT
                if moca.empty or (filekey not in moca[Bucket.FILEKEY].tolist()):
                    df[Bucket.FILEKEY] = filekey
                    df = put_filekey_first(df)
                    moca = pd.concat([moca, df], ignore_index=True)
        
        if pattern=="PDQ8":
            if Qnnrs.PDQ8.value not in row or pd.isna(row[Qnnrs.PDQ8.value]) or row[Qnnrs.PDQ8.value]=='':
                df = download_csv_to_df(filekey)
                new_filekeys.loc[ii, Qnnrs.PDQ8] = df.loc[0, PDQ8.pdq8.value].astype(int).sum()
                new_filekeys.loc[ii, Bucket.SAMPLER] = df.loc[0, Bucket.SAMPLER] if Bucket.SAMPLER in df else pd.NaT
                if pdq8.empty or (filekey not in pdq8[Bucket.FILEKEY].tolist()):
                    df[Bucket.FILEKEY] = filekey
                    df = put_filekey_first(df)
                    pdq8 = pd.concat([pdq8, df], ignore_index=True)
        
        if pattern=="FOG":
            if Qnnrs.FOG.value not in row or pd.isna(row[Qnnrs.FOG.value]) or row[Qnnrs.FOG.value]=='':
                df = download_csv_to_df(filekey)
                new_filekeys.loc[ii, Qnnrs.FOG] = df.loc[0, FOG.fog.value].astype(int).sum()
                if fog.empty or (filekey not in fog[Bucket.FILEKEY].tolist()):
                    df[Bucket.FILEKEY] = filekey
                    df = put_filekey_first(df)
                    fog = pd.concat([fog, df], ignore_index=True)
        
        if pattern=="SDQ":
            if Qnnrs.SDQ.value not in row or pd.isna(row[Qnnrs.SDQ.value]) or row[Qnnrs.SDQ.value]=='':
                df = download_csv_to_df(filekey)
                df = df.replace({"True": 1, "False": 0})
                score = df.loc[0, SDQ.sdq.value[:-1]].astype(int).sum()
                respiratory = 2.5 if df.loc[0, SDQ.sdq.value[-1]]=="True" else 0.5
                score += respiratory
                new_filekeys.loc[ii, Qnnrs.SDQ] = score
                if sdq.empty or (filekey not in sdq[Bucket.FILEKEY].tolist()):
                    df[Bucket.FILEKEY] = filekey
                    df = put_filekey_first(df)
                    sdq = pd.concat([sdq, df], ignore_index=True)
        
        if pattern=="WOQ":
            if Qnnrs.WOQ_PRE.value not in row or pd.isna(row[Qnnrs.WOQ_PRE.value]) or row[Qnnrs.WOQ_PRE.value]=='':
                df = download_csv_to_df(filekey)
                df = df.replace({"True": 1, "False": 0})
                try:
                    new_filekeys.loc[ii, Qnnrs.WOQ_PRE] = df.loc[0, WOQ.pre.value].astype(int).sum()
                    new_filekeys.loc[ii, Qnnrs.WOQ_POST] = df.loc[0, WOQ.pre.value].astype(int).sum() - df.loc[0, WOQ.post.value].astype(int).sum()
                except:
                    new_filekeys.loc[ii, Qnnrs.WOQ_PRE] = -1
                    new_filekeys.loc[ii, Qnnrs.WOQ_POST] = -1
                if woq.empty or (filekey not in woq[Bucket.FILEKEY].tolist()):
                    df[Bucket.FILEKEY] = filekey
                    df = put_filekey_first(df)
                    woq = pd.concat([woq, df], ignore_index=True)

        if pattern=="REGISTRATION":
            if Registration.BIRTHDATE.value not in row or pd.isna(row[Registration.BIRTHDATE.value]) or row[Registration.BIRTHDATE.value]=='':
                df = download_csv_to_df(filekey)
                for col in Registration.values():
                    if col in df:
                        new_filekeys.loc[ii, col] = str(df.loc[0, col])
                if registration.empty or (filekey not in registration[Bucket.FILEKEY].tolist()):
                    df[Bucket.FILEKEY] = filekey
                    df = put_filekey_first(df)
                    registration = pd.concat([registration, df], ignore_index=True)
    
        if pattern=="UPDATE":
            if Update.IS_DBS.value not in row or pd.isna(row[Update.IS_DBS.value]) or row[Update.IS_DBS.value]=='':
                df = download_csv_to_df(filekey)
                for col in Update.values():
                    if col in df:
                        new_filekeys.loc[ii, col] = str(df.loc[0, col])
                if update.empty or (filekey not in update[Bucket.FILEKEY].tolist()):
                    df = download_csv_to_df(filekey)
                    df[Bucket.FILEKEY] = filekey
                    df = put_filekey_first(df)
                    update = pd.concat([update, df], ignore_index=True)

        if pattern=="MEDICATIONS":
            if medications.empty or (filekey not in medications[Bucket.FILEKEY].tolist()):
                df = download_csv_to_df(filekey)
                df[Bucket.FILEKEY] = filekey
                df = put_filekey_first(df)
                medications = pd.concat([medications, df.astype(str)], ignore_index=True)

        
    updrs.to_csv(Settings.UPDRS_CSV, index=False)
    moca.to_csv(Settings.MOCA_CSV, index=False)
    pdq8.to_csv(Settings.PDQ8_CSV, index=False)
    fog.to_csv(Settings.FOG_CSV, index=False)
    sdq.to_csv(Settings.SDQ_CSV, index=False)
    woq.to_csv(Settings.WOQ_CSV, index=False)
    update.to_csv(Settings.UPDATE_CSV, index=False)
    registration.to_csv(Settings.REGISTRATION_CSV, index=False)
    medications.to_csv(Settings.MEDICATION_CSV, index=False)

    raw = pd.concat([raw, new_filekeys], ignore_index=True)
    raw = raw.sort_values(by=['date', 'username', 'time'], ascending=False, ignore_index=True)
    raw.to_csv(Settings.RAW_CSV, index=False)
```

# "Get all files"

* `change_columns()`: update values of attributes (columns) for specific filekeys.
* `resolve()`: remove unneeded filekeys, update attributes for old patterns or filenames.
* `add_patient_phone()`: merge sampler's name (from the registration csv) with samplers phones csv.
* `add_patient_phone()`: use the data from the VM to merge the phone numbers of the patients (users).
* `add_session_to_all()`: find the closest session (in time) for the sampler's files.
* `resolve_sessions()`: resolve sessions issues; mostly merging sessions.
* `propagate_values()`: propagate the data from the csv files to all filekeys.
* Create `all_files.csv`, `sessions.csv`, and `all_users.csv`.


```python
def get_all_files():
    print("Arranging data ...", end=' ')
    df = pd.read_csv(Settings.RAW_CSV, dtype=str)
    df = change_columns(df)
    df = resolve(df)

    df = add_sampler_phone(df)
    df = add_patient_phone(df)
    df = add_caregiver_phone(df)

    df['datetime'] = pd.to_datetime(df['datetime'], format=Settings.DATETIME, errors='coerce')
    df = add_session_to_all(df)

    real_sessions = df.copy()
    real_sessions = add_session_number(real_sessions)
    real_sessions = add_sampler_to_HC(real_sessions)

    df = resolve_sessions(df)
    df = add_session_number(df)
    df = remove_qnnrs_duplicates(df)
    df = add_sampler_to_HC(df)
    print("Done!")

    df = propagate_values(df)
    df = add_age(df)
    df = add_updrs_columns(df)
    df = df.sort_values(by=['date', 'username', 'time'], ascending=False)
    df.to_csv(Settings.ALL_FILES, index=False)
    
    sessions = get_sessions(df)
    sessions.to_csv(Settings.SESSIONS, index=False)
    real_sessions = get_sessions(real_sessions)
    real_sessions.to_csv(Settings.REAL_SESSIONS, index=False)

    all_users = get_all_users(df)
    all_users.to_csv(Settings.ALL_USERS, index=False)
    ```

# Plots

### Pie plots
![](pies.png)

```python
def plot_pies(show_ax=False, exclude_archive=True, only_full=True, counter='time'):
    users = pd.read_csv(Settings.ALL_USERS, dtype=str)
    patients = users[users.entity=='PD']
    healthy = users[users.entity=='HC']
    # if exclude_archive:
    #     healthy = healthy[~healthy.username.isin(IgnoreUsers.archive)]

    npm = len(patients.loc[patients.gender == 'male', 'username'].unique())
    npf = len(patients.loc[patients.gender == 'female', 'username'].unique())
    nhm = len(healthy.loc[healthy.gender == 'male', 'username'].unique())
    nhf = len(healthy.loc[healthy.gender == 'female', 'username'].unique())

    sessions = pd.read_csv(Settings.SESSIONS, dtype=str)
    # sessions = sessions[~sessions.username.isin(Exclude.users)]
    if only_full:
        sessions = sessions[~sessions['paradigm'].str.contains("~")]
    s_pd = len(sessions[(sessions.entity == 'PD')])
    s_hc = len(sessions[(sessions.entity == 'HC')])

    fig, axs = plt.subplots(2, 2, figsize=(12, 14))

    labels = ['PD men', 'PD women', 'HC men', 'HC women']
    sizes = [npm, npf, nhm, nhf]
    colors = [Color.VV, Color.VV, Color.YELLOW, Color.YELLOW]
    explode = [0.03, 0.03, 0.03, 0.03]

    if show_ax:
        ataxia = users[users.entity=='AX']
        na = len(ataxia.username.unique())
        labels.append('Ataxia')
        sizes.append(na)
        colors.append(Color.LIGHT_GREEN)
        explode.append(0.05)

    wedges, texts, autotexts = axs[0, 0].pie(sizes, explode=explode, labels=labels, colors=colors,
                                             autopct=lambda p: f'{round(p * sum(sizes) / 100)}', shadow=True,
                                             startangle=140)
    for autotext in autotexts:
        autotext.set_fontsize(18)
    axs[0, 0].set_title('Participants')



    if counter=='number':
        labels = ['PD', 'HC']
        sizes = [s_pd * 46, s_hc * 23]
        for ii in range(len(labels)):
            labels[ii] = f"{labels[ii]}: {sizes[ii] // 23}"
    elif counter=='time':
        files = pd.read_csv(Settings.ALL_FILES, dtype=str)
        files = files[files['pattern'].isin(['RECORDING','RECORDING1'])]
        files['duration'] = files['exercise'].apply(lambda k: Durations[k])
        labels = ['PD', 'HC']
        sizePD = files.loc[files['entity']=='PD', 'duration'].sum()//60
        sizeHC = files.loc[files['entity']=='HC', 'duration'].sum()//60
        sizes = [sizePD, sizeHC]

    colors = [Color.VV, Color.YELLOW]
    explode = (0.03, 0.03)

    wedges, texts, autotexts = axs[0, 1].pie(sizes, explode=explode, labels=labels, colors=colors,
                                             autopct=lambda p: f'{round(p * sum(sizes) / 100)}\nminutes', shadow=True,
                                             startangle=140)
    for autotext in autotexts:
        autotext.set_fontsize(18)
        # autotext.set_color('white')
    axs[0, 1].set_title('Recordings')



    n_sheba = len(patients.loc[patients.medical_center == 'Sheba', 'username'].unique())
    n_ichilov = len(patients.loc[patients.medical_center == 'Ichilov', 'username'].unique())

    labels = ['Sheba', 'Ichilov']
    sizes = [n_sheba, n_ichilov]
    colors = [Color.DARK_VV, Color.GREEN]
    explode = (0.03, 0.03)

    wedges, texts, autotexts = axs[1, 0].pie(sizes, explode=explode, labels=labels, colors=colors,
                                             autopct=lambda p: f'{round(p * sum(sizes) / 100)}', shadow=True,
                                             startangle=140)
    for autotext in autotexts:
        autotext.set_color('white')
        autotext.set_fontsize(18)
    axs[1, 0].set_title('Medical Centers')

    n_no = len(patients.loc[patients.genetic == 'NO', 'username'].unique())
    n_yes = len(patients.loc[patients.genetic == 'YES', 'username'].unique())
    n_gba = len(patients.loc[patients.genetic == 'GBA', 'username'].unique())
    n_lrrk2 = len(patients.loc[patients.genetic == 'LRRK2', 'username'].unique())

    labels = ['No', 'Yes', 'GBA', 'LRRK2']
    sizes = [n_no, n_yes, n_gba, n_lrrk2]
    colors = [Color.VV, Color.LIGHT_GREEN, Color.YELLOW, Color.GREEN]
    explode = (0.03, 0.03, 0.03, 0.03)

    wedges, texts, autotexts = axs[1, 1].pie(sizes, explode=explode, labels=labels, colors=colors,
                                             autopct=lambda p: f'{round(p * sum(sizes) / 100)}', shadow=True,
                                             startangle=140)
    for autotext in autotexts:
        autotext.set_fontsize(18)
    axs[1, 1].set_title('Genetic predisposition')

    return fig, axs
    ```

### Users over time
![](overtime.png)

```python
def plot_users_over_time(interval=None, exclude_archive=True, dropout=False, annotation_percentage=20):
    users = pd.read_csv(Settings.ALL_USERS, dtype=str)
    patients = users[users.entity == 'PD']
    healthy = users[users.entity == 'HC']
    
    # if exclude_archive:
    #     healthy = healthy[~healthy.username.isin(IgnoreUsers.archive)]

    patients['date'] = pd.to_datetime(patients['date'])
    healthy['date'] = pd.to_datetime(healthy['date'])

    if dropout:
        dropout_df = pd.read_csv(Settings.DROPOUT, dtype=str)
        dropout_df['drop_out'] = pd.to_datetime(dropout_df['drop_out'], format=Settings.DATE_FORMAT)

    patients_counts = patients.groupby(patients['date'].dt.date).size().cumsum()
    healthy_counts = healthy.groupby(healthy['date'].dt.date).size().cumsum()

    combined_counts_patients = pd.DataFrame({'Patients': patients_counts})
    combined_counts_healthy = pd.DataFrame({'Healthy': healthy_counts})

    if dropout:
        # Convert all relevant dates to the same type (Timestamp)
        all_dates = pd.date_range(start=min(combined_counts_patients.index.min(), dropout_df['drop_out'].min().date()), 
                                  end=max(combined_counts_patients.index.max(), dropout_df['drop_out'].max().date()))
        combined_counts_patients = combined_counts_patients.reindex(all_dates, method='ffill').fillna(0)
        combined_counts_healthy = combined_counts_healthy.reindex(all_dates, method='ffill').fillna(0)

        for _, row in dropout_df.iterrows():
            username = hashp(row['user_phone'])
            if username in patients.username.values:
                drop_date = row['drop_out'].date()
                combined_counts_patients.loc[drop_date:] -= 1
            elif username in healthy.username.values:
                drop_date = row['drop_out'].date()
                combined_counts_healthy.loc[drop_date:] -= 1

    if interval:
        start_date, end_date = pd.to_datetime(interval).date
        combined_counts_patients = combined_counts_patients[(combined_counts_patients.index >= start_date) & (combined_counts_patients.index <= end_date)]
        combined_counts_healthy = combined_counts_healthy[(combined_counts_healthy.index >= start_date) & (combined_counts_healthy.index <= end_date)]

    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(combined_counts_patients.index, combined_counts_patients['Patients'], color=Color.VV, label='Patients', marker='o', alpha=0.7)
    ax.plot(combined_counts_healthy.index, combined_counts_healthy['Healthy'], color=Color.YELLOW, label='Healthy', marker='o', alpha=0.7)

    fig.autofmt_xdate()

    step_patients = max(1, len(combined_counts_patients) * annotation_percentage // 100)
    step_healthy = max(1, len(combined_counts_healthy) * annotation_percentage // 100)

    for i, (date, row) in enumerate(combined_counts_patients.iterrows()):
        if i % step_patients == 0:
            ax.annotate(f'{row["Patients"]:.0f}', (date, row['Patients']), textcoords="offset points", xytext=(0, 10), ha='center', color=Color.VV)

    for i, (date, row) in enumerate(combined_counts_healthy.iterrows()):
        if i % step_healthy == 0:
            ax.annotate(f'{row["Healthy"]:.0f}', (date, row['Healthy']), textcoords="offset points", xytext=(0, 10), ha='center', color=Color.YELLOW)

    upper_limit = 1.2 * max(combined_counts_healthy.max().max(), combined_counts_patients.max().max())
    ax.set_ylim([0, upper_limit])
    ax.set_xlabel('Date')
    ax.set_ylabel('Cumulative Number of Users')
    ax.set_title('Number of Users Joined Over Time')
    ax.legend(loc='lower right')
    ax.grid(True)
    return fig, ax
```

### Users per sampler
![How many users (PD/HC) each sampler sampled?](userpersampler.png)

```python
def users_per_sampler(exclude_archive=True, show=None, ignore=[]):
    import pandas as pd
    import matplotlib.pyplot as plt

    users = pd.read_csv(Settings.ALL_USERS, dtype=str)
    users['date'] = pd.to_datetime(users['date'])
    users['month'] = users['date'].dt.to_period('M')
    patients = users[users.entity == 'PD']
    healthy = users[users.entity == 'HC']
    if exclude_archive:
        healthy = healthy[~healthy.username.isin(SpecialUsers.values())]

    samplers = pd.read_csv(Settings.SAMPLERS_CSV, dtype=str)
    if show:
        samplers = samplers.loc[samplers.group == show, 'sampler_username']
        patients = patients[patients['sampler_username'].isin(samplers)]
        healthy = healthy[healthy['sampler_username'].isin(samplers)]

    if ignore:
        patients = patients[~patients['sampler_username'].isin(ignore)]
        healthy = healthy[~healthy['sampler_username'].isin(ignore)]
    
    patient_counts = patients.groupby(['Hebrew', 'month'])['username'].nunique().unstack(fill_value=0)
    healthy_counts = healthy.groupby(['Hebrew', 'month'])['username'].nunique().unstack(fill_value=0)
    patient_cumulative = patient_counts.cumsum(axis=1)
    patient_cumulative = patient_cumulative.mask(patient_cumulative.diff(axis=1) == 0)

    combined_counts = pd.DataFrame({
        'Patients': patient_counts.sum(axis=1),
        'Healthy': healthy_counts.sum(axis=1)
    }).fillna(0)

    fig, ax = plt.subplots(figsize=(12, 6))
    width = 0.4

    combined_counts['Patients'].plot(kind='bar', color=Color.VV, width=width, position=1, label='Patients', ax=ax)
    combined_counts['Healthy'].plot(kind='bar', color=Color.YELLOW, width=width, position=0, label='Healthy', ax=ax)

    # Add white horizontal lines for monthly registrations in the PD bars, and month codes below them
    for i, (sampler, row) in enumerate(patient_cumulative.iterrows()):
        for month in range(1, len(row)):
            y_position = row.iloc[month-1]
            # Draw the white line if it's not at the top or bottom
            if y_position > 0: # and y_position < row.iloc[-1]:
                # Draw the white line only on the PD bar
                ax.plot([i - width, i], [y_position]*2, color='white', linewidth=1.5)
                # Add the month code below the white line
                month_code = row.index[month-1].strftime('%b')
                ax.text(i - width/2, y_position - 0.3, month_code, ha='center', color='white', va='top', fontsize=8)

    ax.set_xlabel('Sampler')
    ax.set_ylabel('Number of Users')
    if show:
        ax.set_title(f'Number of Users per Sampler - {show}')
    else:
        ax.set_title('Number of Users per Sampler')
    ax.set_xticklabels([label.get_text()[::-1] for label in ax.get_xticklabels()], rotation=45)

    for i, (patients, healthy) in enumerate(zip(combined_counts['Patients'], combined_counts['Healthy'])):
        ax.text(i - width/2, patients + 1.0, f'{patients:.0f}', ha='center', color='black', va='bottom', fontsize=12)
        ax.text(i + width/2, healthy + 1.0, f'{healthy:.0f}', ha='center', color='black', va='bottom', fontsize=12)

    ax.legend()
    ax.grid(axis='y')
    ax.set_xlim([-1, len(combined_counts)])
    ax.set_ylim([0, 1.2 * combined_counts.max().max()])

    return fig, ax
    ```

### Number of sessions vs. session number
![](sessions.png)

```python
def plot_sessions_count(n, show_all=True, exclude_archive=True):
    sessions = pd.read_csv(Settings.SESSIONS, dtype=str)
    if exclude_archive:
        sessions = sessions[~sessions.username.isin(SpecialUsers.values())]
    sessions = sessions[(sessions['entity'] != 'AX')]

    all_table = pd.DataFrame(index=range(1, n+1), columns=sessions['entity'].unique())
    for i in all_table.index:
        for entity in all_table.columns:
            count = sessions[(sessions['entity'] == entity) & (sessions['session_number'].astype(int) >= i)]['username'].nunique()
            all_table.at[i, entity] = count

    full_sessions = sessions[(~sessions.paradigm.str.contains("~"))]
    full_table = pd.DataFrame(index=range(1, n+1), columns=full_sessions['entity'].unique())
    for i in full_table.index:
        for entity in full_table.columns:
            count = full_sessions[(full_sessions['entity'] == entity) & (full_sessions['session_number'].astype(int) >= i)]['username'].nunique()
            full_table.at[i, entity] = count

    # Load dropout data
    dropout = pd.read_csv(Settings.DROPOUT, dtype=str)
    first_bar_value = all_table['PD'].iloc[0]
    active_users = first_bar_value - len(dropout)

    fig, ax = plt.subplots(figsize=(12, 6))

    bar_width = 0.35
    indices = range(len(all_table.index))

    if show_all:
        ax.bar([i - bar_width/2 for i in indices], all_table['PD'], bar_width, label='All PD', color=Color.VV, alpha=0.5)
        ax.bar([i + bar_width/2 for i in indices], all_table['HC'], bar_width, label='All HC', color=Color.YELLOW, alpha=0.5)

    ax.bar([i - bar_width/2 for i in indices], full_table['PD'], bar_width, label='Full PD', color=Color.VV)
    ax.bar([i + bar_width/2 for i in indices], full_table['HC'], bar_width, label='Full HC', color=Color.YELLOW)

    if show_all:
        for i, v in enumerate(all_table['PD']):
            if v == 0:
                continue
            ax.text(i - bar_width/2, v + 4, str(v), ha='center', va='top', color='black', fontsize=12)
        for i, v in enumerate(all_table['HC']):
            if v == 0:
                continue
            ax.text(i + bar_width/2, v + 4, str(v), ha='center', va='top', color='black', fontsize=12)

    for i, v in enumerate(full_table['PD']):
        if v == 0:
            continue
        ax.text(i - bar_width/2, v - 4, str(v), ha='center', va='bottom', color='white', fontsize=12)
    for i, v in enumerate(full_table['HC']):
        if v == 0:
            continue
        ax.text(i + bar_width/2, v - 4, str(v), ha='center', va='bottom', color='white', fontsize=12)

    ax.axhline(y=active_users, color=Color.GREEN, linewidth=1, linestyle='--')
    ax.set_yticks([active_users])
    # ax.set_yticklabels([f'Active Users: {active_users}'], fontsize=12, color='pink')

    ax.set_xlabel('Number of Sessions')
    ax.set_ylabel('Number of Users')
    ax.set_title(f'Number of Users with up to {n} Sessions')
    ax.legend(title='Entity')
    ax.grid(axis='y')
    ax.set_xticks(indices)
    ax.set_xticklabels(all_table.index, rotation=0)
    
    return fig, ax
    ```

### Questionnaires results
![](qnnrs.png)

```python
def plot_histograms(bins=16):
    # Load the data
    data = pd.read_csv(Settings.ALL_FILES, dtype=str)
    exercises_order = ['updrs3', 'updrs124', 'moca', 'fog', 'sdq', 'woq', 'pdq8']
    
    # Create subplots
    fig, axes = plt.subplots(3, 3, figsize=(18, 18))
    axes = axes.flatten()
    
    # Hide all axes initially
    for ax in axes:
        ax.axis('off')
    
    for idx, exercise in enumerate(exercises_order):
        exercise_data = data[data['exercise'] == exercise]
        
        if exercise == 'updrs3':
            pre_scores = exercise_data[exercise_data['timing'] == 'pre']['updrs3_pre'].dropna().astype(float)
            post_scores = exercise_data[exercise_data['timing'] == 'post']['updrs3_post'].dropna().astype(float)
            
            axes[idx].hist(pre_scores, bins=np.linspace(0,132,bins[idx]) if isinstance(bins, list) else bins, alpha=0.5, label='Pre', color=Color.VV)
            axes[idx].hist(post_scores, bins=np.linspace(0,132,bins[idx]) if isinstance(bins, list) else bins, alpha=0.5, label='Post', color=Color.YELLOW)
            axes[idx].set_xlabel('Score')
            axes[idx].set_ylabel('Frequency')
            axes[idx].set_title('UPDRS3 Scores')
            axes[idx].legend(loc='upper right')
        
        elif exercise == 'updrs124':
            scores = exercise_data[['updrs1', 'updrs2', 'updrs4']].dropna().astype(float)
            summed_scores = scores.apply(sum,1)
            
            axes[idx].hist(summed_scores, bins=bins[idx] if isinstance(bins, list) else bins, color=Color.GREEN)
            axes[idx].set_xlabel('Summed Score')
            axes[idx].set_ylabel('Frequency')
            axes[idx].set_title('UPDRS124 Summed Scores')
        
        elif exercise == 'woq':
            pre_scores = exercise_data['woq_pre'].dropna().astype(float)
            post_scores = exercise_data['woq_post'].dropna().astype(float)
                        
            axes[idx].hist(pre_scores, bins=bins[idx] if isinstance(bins, list) else bins, alpha=0.5, label='Pre', color=Color.VV)
            axes[idx].hist(post_scores, bins=bins[idx] if isinstance(bins, list) else bins, alpha=0.5, label='Post', color=Color.YELLOW)
            axes[idx].set_xlabel('Score')
            axes[idx].set_ylabel('Frequency')
            axes[idx].set_title('WOQ Scores')
            axes[idx].legend(loc='upper right')
        
        else:
            scores = exercise_data[exercise].dropna().apply(pd.to_numeric, errors='coerce').dropna()
            
            axes[idx].hist(scores, bins=bins[idx] if isinstance(bins, list) else bins, color=Color.GREEN)
            axes[idx].set_xlabel('Score')
            axes[idx].set_ylabel('Frequency')
            axes[idx].set_title(f'{exercise.upper()} Scores')
        
        axes[idx].axis('on')
    
    fig.tight_layout()
    return fig, axes
    ```

### Uncompleted session (broken sessions)
![](broken.png)

```python
def broken_sessions(timeframe=60, only_sampler=False, ignore_sessions=True, fontsize=12, scale=(1.5, 3.1), figsize=(20, 10)):
    df = pd.read_csv(Settings.SESSIONS, dtype=str)
    if ignore_sessions:        
        resolved = pd.read_csv(Settings.RESOLVED_SESSIONS, dtype=str)
        filtered_sessions_df = df.merge(
        resolved[['username', 'session']],
        on=['username', 'session'],
        how='left',
        indicator=True
        )
        filtered_sessions_df = filtered_sessions_df[filtered_sessions_df['_merge'] == 'left_only']
        df = filtered_sessions_df.drop(columns=['_merge'])
        
        dropout = pd.read_csv(Settings.DROPOUT, dtype=str)
        dropout['username'] = dropout['user_phone'].apply(hashp)
        dropout = dropout['username'].tolist()
        df = df[~df['username'].isin(dropout)].reset_index(drop=True)

    samplers = pd.read_csv(Settings.SAMPLERS_CSV, dtype=str, usecols=['sampler_username', 'Hebrew'])

    # Merge samplers dataframe with sessions dataframe
    df = df.merge(samplers, on='sampler_username', how='left')
    df.rename(columns={'Hebrew': 'sampler'}, inplace=True)
    # df.rename(columns={'sampler_username': 'sampler'}, inplace=True)
    df['sampler'] = df['sampler'].fillna(df['session_number'])

    broken_sessions = df.loc[df['paradigm'].str.contains("~"), ['date', 'username', 'session', 'user_phone', 'start', 'end', 'sampler', 'paradigm']]
    broken_sessions.to_csv(Settings.BROKEN_SESSIONS, index=False)
    
    # Show only last 30 days
    broken_sessions['date'] = pd.to_datetime(broken_sessions['date'], format=Settings.DATE_FORMAT)
    days_ago = datetime.now() - timedelta(days=timeframe)
    broken_sessions = broken_sessions.loc[broken_sessions['date'] >= days_ago]
    broken_sessions['date'] = broken_sessions['date'].dt.strftime(Settings.DATE_FORMAT)
    
    # Ensure columns are strings before checking for "~"
    broken_sessions['paradigm'] = broken_sessions['paradigm'].astype(str).str.replace("'","")

    # broken_sessions = df.loc[df['paradigm'].str.contains("~"), ['date', 'username', 'session', 'user_phone', 'start', 'end', 'sampler', 'paradigm']]
    # broken_sessions.to_csv(Settings.BROKEN_SESSIONS, index=False)

    broken_sessions = broken_sessions[['date', 'user_phone', 'start', 'end', 'sampler', 'paradigm']]
    broken_sessions['user_phone'] = broken_sessions['user_phone'].apply(lambda x: x[4:])
    if only_sampler:
        broken_sessions = broken_sessions[~broken_sessions['sampler'].str.isdigit()]

    fig, ax = plt.subplots(figsize=figsize)  # Increase figure size for better fitting
    ax.axis('tight')
    ax.axis('off')

    # Escape special characters in column headers and data, and replace 'nan' with empty strings
    broken_sessions.columns = [str(col).replace('$', r'\$') for col in broken_sessions.columns]
    broken_sessions = broken_sessions.fillna('')  # Replace NaN with empty strings
    broken_sessions = broken_sessions.apply(lambda x: x.map(lambda y: str(y).replace('$', r'\$')))
    broken_sessions = broken_sessions.astype(str).map(lambda x: '' if x == 'nan' else x.replace('$', r'\$'))

    column_widths = {
        'date': 0.085,
        'user_phone': 0.09,
        'start': 0.08,
        'end': 0.08,
        'sampler': 0.1,
        'paradigm': 0.38,
    }

    # Create the table with some formatting
    table = ax.table(cellText=broken_sessions.values, colLabels=broken_sessions.columns, cellLoc='center', loc='center')
    table.auto_set_font_size(False)
    table.set_fontsize(fontsize)
    table.scale(*scale)  # Adjust scale to make sure columns are wide enough

    cell_dict = table.get_celld()
    # Adjust column widths based on the provided dictionary
    for j, col in enumerate(broken_sessions.columns):
        col_width = column_widths.get(col, 0.1)  # Default width if not specified in the dictionary
        for i in range(len(broken_sessions) + 1):
            cell_dict[(i, j)].set_width(col_width)

    # Add colors to the table and handle special cases
    for i in range(len(broken_sessions) + 1):
        for j in range(len(broken_sessions.columns)):
            cell_dict[(i, j)].set_edgecolor('black')
            cell_dict[(i, j)].set_linewidth(1)
            if i == 0:
                cell_dict[(i, j)].set_facecolor('#40466e')
                cell_dict[(i, j)].set_text_props(weight='bold', color='white')
            else:
                if '~' in cell_dict[(i, j)].get_text().get_text():
                    cell_dict[(i, j)].set_facecolor('#bebfd4')  # Very light purple
                else:
                    cell_dict[(i, j)].set_facecolor('#f0f0f0' if i % 2 == 0 else '#e0e0e0')


    # Here's where we apply the row border coloring
    sampler_col_index = broken_sessions.columns.get_loc('sampler')
    user_paradigm_col_index = broken_sessions.columns.get_loc('paradigm')
    duplicate_phones = broken_sessions['user_phone'].duplicated(keep=False)

    color_map = {
                2: '#ff0000',     # Red
                3: '#90ee90',     # Light green
                4: '#ff00ff',     # Magenta
                5: '#ffff00',     # Yellow
                6: '#006400',     # Dark green
                7: '#000000',     # Black
                8: '#a52a2a',     # Brown
                9: '#0000ff',     # Blue
                10: '#ffa500'     # Orange
            }
    
    for i in range(1, len(broken_sessions) + 1):
        if broken_sessions.iloc[i - 1]['sampler'].isdigit():
            sampler_value = int(broken_sessions.iloc[i - 1]['sampler'])
            color = color_map.get(sampler_value, '#ffffff')  # Default to white if not in map

            # Apply the color with half transparency
            cell_dict[(i, sampler_col_index)].set_facecolor(color)
            cell_dict[(i, sampler_col_index)].set_alpha(0.5)
            # Color borders for 'sampler' column
            # cell_dict[(i, sampler_col_index)].set_edgecolor('#3232ff')
            # cell_dict[(i, sampler_col_index)].set_linewidth(4)
            # Color borders for 'user_paradigm' column
            # cell_dict[(i, user_paradigm_col_index)].set_edgecolor('#3232ff')
            # cell_dict[(i, user_paradigm_col_index)].set_linewidth(4)

        if duplicate_phones.iloc[i - 1]:
        # Get the corresponding sampler value
            sampler_value = broken_sessions.iloc[i - 1]['sampler']
            if sampler_value.isdigit():
                sampler_value = int(sampler_value)
                color = color_map.get(sampler_value, '#ffffff')  # Default to white if not in map

                # Apply the color with half transparency to the 'user_phone' cell
                cell_dict[(i, broken_sessions.columns.get_loc('user_phone'))].set_facecolor(color)
                cell_dict[(i, broken_sessions.columns.get_loc('user_phone'))].set_alpha(0.5)


    # Ensure 'sampler' text is right-to-left
    for i in range(1, len(broken_sessions) + 1):
        for j in range(len(broken_sessions.columns)):
            if broken_sessions.columns[j] == 'sampler':
                cell_dict[(i, j)].get_text().set_text(' ' + cell_dict[(i, j)].get_text().get_text()[::-1])  # Reverse text for RTL effect

    # Ensure 'user_paradigm' text is aligned to the left and uses a monospaced font
    for i in range(1, len(broken_sessions) + 1):
        for j in range(len(broken_sessions.columns)):
            if broken_sessions.columns[j] in ['paradigm']:
                cell_dict[(i, j)].get_text().set_ha('center')
                cell_dict[(i, j)].get_text().set_fontproperties(font_manager.FontProperties(family='monospace', size=fontsize))
    
    return fig, ax
    ```

### Users that need to record their next session
![](password.png)

```python
def plot_dataframe_table(source_file, X=28, fontsize=11, scale=1.3, figsize=(20, 24)):
    
    def send_recording_update_to(source_file=source_file, X=X):
        with open(Settings.CONFIG, 'r') as file:
            cfg = yaml.safe_load(file)

        # Load session and dropout data
        sessions_df = pd.read_csv(Settings.SESSIONS, dtype=str)
        dropout = pd.read_csv(Settings.DROPOUT, dtype=str)
        dropout['username'] = dropout['user_phone'].apply(hashp)
        dropout = dropout['username'].tolist()
        sessions_df = sessions_df[~sessions_df['username'].isin(dropout)].reset_index(drop=True)

        # Filter by 'entity' == 'PD'
        sessions_df = sessions_df[sessions_df['entity'] == 'PD']

        # Convert 'date' to datetime for sorting and calculation purposes
        sessions_df['date'] = pd.to_datetime(sessions_df['date'])

        # Keep only the last session for each user
        sessions_df = sessions_df.sort_values(by=['username', 'date'], ascending=[True, False])
        sessions_df = sessions_df.drop_duplicates(subset='username', keep='first').reset_index(drop=True)
        
        # Rename 'date' to 'last session' and calculate 'days'
        sessions_df['last session'] = sessions_df['date']
        sessions_df['days'] = sessions_df['last session'].apply(lambda x: (datetime.now() - x).days)
        
        # Sort by 'days'
        sessions_df = sessions_df.sort_values(by='days', ascending=False, ignore_index=True)
        
        # Add 'send?' column based on the X value
        sessions_df['SMS'] = sessions_df['days'].apply(lambda x: 1 if x >= X else 0)
        sessions_df = sessions_df[sessions_df['days'] >= X-3]
        
        # Check if the source file exists
        if os.path.exists(source_file):
            # Add columns for 'called', 'answered', and 'quit' from source_file
            source_df = pd.read_csv(source_file, dtype=str)
            
            # Check if 'called', 'answered', and 'quit' exist in the source file, if not, set NaN
            for col in cfg['gal_columns']:
                if col not in source_df.columns:
                    source_df[col] = np.nan
            
            # Merge source_df with sessions_df on 'username'
            sessions_df = pd.merge(sessions_df, source_df[['username'] + cfg['gal_columns']], on='username', how='left')
        else:
            # If the source file doesn't exist, add NaN for 'called', 'answered', and 'quit'
            sessions_df['called'] = np.nan
            sessions_df['answered'] = np.nan
            sessions_df['quit'] = np.nan
        
        # Reorder columns to include new ones
        sessions_df = sessions_df[['username', 'user_phone', 'password', 'last session', 'days', 'session_number', 'SMS']+cfg['gal_columns']]
        
        return sessions_df


    df = send_recording_update_to(source_file)  
    # Replace 'NaN' with an empty string
    df = df.fillna('')

    # Adjust 'last session' to show only the date
    if 'last session' in df.columns:
        df['last session'] = pd.to_datetime(df['last session']).dt.date

    # Export the DataFrame to an Excel file
    df.to_csv(Settings.SEND_PASSWORDS, index=False)

    df['comments'] = df['comments'].apply(lambda x: x[::-1] if isinstance(x, str) else x)
    df.rename(columns={'session_number': 'number',
                       'user_phone': 'phone'}, inplace=True)

    fig, ax = plt.subplots(figsize=figsize)  # Increase figure size for better fitting
    ax.axis('tight')
    ax.axis('off')

    # Escape special characters in column headers and data
    df.columns = [str(col).replace('$', r'\$') for col in df.columns]
    df = df.apply(lambda x: x.map(lambda y: str(y).replace('$', r'\$')))
    df['phone'] = df['phone'].apply(lambda x: x[4:])
    # Create the table with some formatting
    table = ax.table(cellText=df.values, colLabels=df.columns, cellLoc='center', loc='center')
    table.auto_set_font_size(False)
    table.set_fontsize(fontsize)
    table.scale(scale, scale+0.5)  # Adjust scale to make sure columns are wide enough

    # Format the 'username' column to be bold
    cell_dict = table.get_celld()
    for i in range(len(df)):
        cell_dict[(i + 1, 0)].set_text_props(weight='bold')

    # Adjust column widths
    for j in range(len(df.columns)):
        max_len = max([len(str(s)) for s in df.iloc[:, j]] + [len(df.columns[j])])
        table.auto_set_column_width(j)
        if max_len > 10:
            table.auto_set_column_width(j)
            for i in range(len(df) + 1):
                cell_dict[(i, j)].set_width(0.15)

    # Add colors to the table
    for i in range(len(df) + 1):
        for j in range(len(df.columns)):
            cell_dict[(i, j)].set_edgecolor('black')
            cell_dict[(i, j)].set_linewidth(1)
            if i == 0:
                cell_dict[(i, j)].set_facecolor('#40466e')
                cell_dict[(i, j)].set_text_props(weight='bold', color='white')
            else:
                cell_dict[(i, j)].set_facecolor('#f0f0f0' if i % 2 == 0 else '#e0e0e0')

    return fig, ax
    ```